# SENTIMENT ANALYZER ENGINE (MODEL)

You need a developer account and a registered App on Twitter for this model.

In [1]:
import tweepy, re
from textblob import TextBlob
import matplotlib.pyplot as plt
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
# Twitter authentication with tweepy

consumerKey = ''
consumerSecret = ''
accessToken = ''
accessTokenSecret = ''

auth = tweepy.OAuthHandler(consumerkey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)

api = tweepy.API(auth)

In [ ]:
# input for term to be searched and how many tweets to search

searchTerm = input("Enter Keyword/Tag to search: ")
NoOfTerms = int(input("How many tweets to search: "))

tweets = []
tweetText = []

# searching for tweets
tweets = tweepy.Cursor(api.search, q=searchTerm+" -filter:retweets", lang="en").items(NoOfTerms)

In [ ]:
tweet_list = [tweet.text for tweet in tweets]
tweet_df = pd.DataFrame(tweet_list)
tweet_df

# Data cleaning

In [2]:
def clean_data(text):
    return ' '.join(re.sub("(@[a-zA-Z0-9]+)|([^0-9A-Za-z])|(https://[\w.]+[\w]+)", " ", text).split())

tweet_df['cleaned_data'] = tweet_df[0].apply(clean_data)

In [ ]:
tweet_df.head()

In [ ]:
def drop_numbers(list_text):
    list_text_new = []
    for i in list_text:
        if not re.search('\d', i):
            list_text_new.append(i)
    return ''.join(list_text_new)

tweet_df['cleaned_data'] = tweet_df['cleaned_data'].apply(drop_numbers)

In [ ]:
tweet_df.head()

In [ ]:
def lower_case(text):
    text_words = word_tokenize(text)
    text_words_lower = [x.lower() for x in text_words]
    return ' '.join(text_words_lower)

tweet_df['cleaned_data'] = tweet_df['cleaned_data'].apply(lower_case)

# Lemmatization

#### FIRST DOWNLOAD THE "WORDNET" PACKAGE FROM "NLTK LIBRARY"

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatise(text):
    text_tokens = word_tokenize(text)
    text_lemm = [lemmatizer.lemmatize(word) for word in text_tokens]
    return ' '.join(text_lemm)

tweet_df['cleaned_data'] = tweet_df['cleaned_data'].apply(lemmatise)

In [ ]:
tweet_df['cleaned_data'].values

### Remove Stopwords

In [ ]:
def remove_stopword(text):
    text_tokens = word_tokenize(text)
    tokens = [word for word in text_tokens if not word in set(stopwords.words('english'))]
    tokens_text = ' '.join(tokens)
    return tokens_text

tweet_df['cleaned_data'] = tweet_df['cleaned_data'].apply(remove_stopword)

In [ ]:
tweet_df['cleaned_data'].values

# Sentiment Analyzer Engine

In [ ]:
# calculate the polarity
# Polarity determines the sentimental aspect of an opinion from -1 to +1,
# where 0 is neutral sentiment

def get_polarity(text):
    textblob = TextBlob(str(text))
    pol = textblob.sentiment.polarity
    if (pol == 0):
        return "Neutral"
    elif (pol>0 and pol <=0.3):
        return "Weakly Positive"
    elif (pol>0.3 and pol <=0.6):
        return "Positive"
    elif (pol>0.6 and pol<=1):
        return "Strongly Positive"
    elif (pol>-0.3 and pol<=0):
        return "Weakly Negative"
    elif (pol>-0.6 and pol<=-0.3):
        return "Negative"
    elif (pol>-1 and pol<=-0.6):
        return "Strongly Negative"
    
tweet_df['polarity'] = tweet_df['cleaned_data'].apply(get_polarity)

In [ ]:
tweet_df['polarity'].value_counts()

In [ ]:
neutral = 0
wpositive = 0
spositive = 0
positive = 0
negative = 0
wnegative = 0
snegative = 0
polarity = 0

for i in range(0,70):
    textblob = TextBlob(str(tweet_df['cleaned_data'][i]))
    polarity += textblob.sentiment.polarity
    pol = textblob.sentiment.polarity
    if (pol==0): #adding reaction of how people are reacting to find
        neutral += 1
    elif (pol>0 and pol<=0.3):
        wpositive += 1
    elif (pol>0.3 and pol<=0.6):
        positive += 1
    elif (pol>0.6 and pol<1):
        spositive +=1
    elif (pol>-0.3 and pol<=0):
        wnegative += 1
    elif (pol>-0.6 and pol<=-0.3):
        negative += 1
    elif (pol>-1 and pol<=-0.6):
        snegative += 1

In [ ]:
# find average reaction

polarity = polarity / noOfTerms
polarity

In [ ]:
def percentage(part, whole):
    temp = 100 * float(part) / float(whole)
    return format(temp, '.2f')

In [ ]:
# finding average of how people are reacting
positive = percentage(positive, NoOfTerms)
wpositive = percentage(wpositive, NoOfTerms)
spositive = percentage(spositive, NoOfTerms)
negative = percentage(negative, NoOfTerms)
wnegative = percentage(wnegative, NoOfTerms)
snegative = percentage(snegative, NoOfTerms)
neutral = percentage(neutral, NoOfTerms)

# Visualizing Results

In [ ]:
# printing data

print("How people are reacting on " + searchTerm + " by analyzing " + str(NoOfTerms) + " tweets.")
print()
print("-----------------------------------------------------------------------------------------")
print()
print("General Report: ")

if (polarity == 0):
    print("Neutral")
elif (polarity > 0 and polarity <= 0.3):
    print("Weakly Positive")
elif (polarity > 0 and polarity <= 0.3):
    print("Positive")
elif (polarity > 0 and polarity <= 0.3):
    print("Strongly Positive")
elif (polarity > 0 and polarity <= 0.3):
    print("Weakly Negative")
elif (polarity > 0 and polarity <= 0.3):
    print("Negative")
elif (polarity > 0 and polarity <= 0.3):
    print("Strongly Positive")
    
print()
print("-----------------------------------------------------------------------------------------")
print()
print("Detailed Report: ")
print(str(positive) + "% people thought it was positive")
print(str(wpositive) + "% people thought it was weakly positive")
print(str(spositive) + "% people thought it was strongly positive")
print(str(negative) + "% people thought it was negaive")
print(str(wnegative) + "% people thought it was weakly negative")
print(str(snegative) + "% people thought it was strongly negative")
print(str(neutral) + "% people thought it was neutral")

In [ ]:
sizer = [positive, wpositive, spositive, neutral, negative, wnegative, snegative]
colors = ['yellowgreen', 'lightgreen', 'darkgreen', 'gold', 'red', 'lightsalmon', 'darkred']
labels = ['Positive [' + str(positive) + '%]', 'Weakly Positive [' + str(wpositive) + '%]',
          'Strongly Positive [' + str(spositive) + '%]', 'Neutral [' + str(neutral) + '%]',
          'Negative [' + str(negative) + '%]', 'Weakly Negative [' + str(wnegative) + '%]',
          'Strongly Negative [' + str(snegative) + '%]']

In [ ]:
plt.pie(sizes, labels = labels, colors = colors)
plt.legend(labels, loc="best")
plt.title('How people are reacting to ' + searchTerm + ' by analyzing ' + str(NoOfTerms) + ' Tweets.')
plt.axis('equal')
plt.tight_layout()
plt.show()